In [3]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
from subprocess import call

In [4]:
%%time

sim = rebound.Simulation.from_file("trappist.bin")
ps = sim.particles

movie=True
Norbits = 1 # of outer planet
Nsec = 1 # length of resulting movie
scale=ps[1].a/5 # length scale for making dots bigger

if movie:
    call("rm -f pngs/*", shell=True)

lw = 3
fps = 30

sim.t = 0
sim.dt = ps[1].P/100
tmax = Norbits*ps[-1].P/sim.dt

Nfigs = Nsec*fps
times = np.linspace(0, ps[-1].P*Norbits, Nfigs)

yprev = np.zeros(sim.N)
refsize=25*lw # this is what REBOUND uses for size of circles in call to plt.scatter
with open("times.txt", "w") as f:
    for i, time in enumerate(times):
        while sim.t < time:
            sim.step()
            for j in range(1,sim.N):
                if yprev[j] < 0 and ps[j].y > 0:
                    f.write("{0:6f}\t{1}\n".format(sim.t, j))
                yprev[j] = ps[j].y
        if movie:
            fig = rebound.OrbitPlot(sim, figsize=(8,8), color=True, lw=lw)
            ax = fig.axes[0]
            ax.axis('off')
            for p in ps[1:]:
                if p.x > 0:
                    ax.scatter(p.x, p.y, s=refsize*(1+5*np.exp(-np.abs(p.y)/scale)),color='k', marker='o')
            fig.savefig('pngs/trappist{0:0=3d}.png'.format(i), transparent=True)
            plt.close(fig)

/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:374: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/Users/dtamayo/Documents/workspace/rebound/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


CPU times: user 5.79 s, sys: 66.4 ms, total: 5.85 s
Wall time: 6.01 s


In [41]:
filename = "test.mp4"
try:
    call("rm -f {0}".format(filename), shell=True)
except:
    pass
call("ffmpeg -r {0} -i pngs/trappist%03d.png -c:v libx264 -pix_fmt yuv420p {1}".format(fps, filename), shell=True)

0